這份 Notebook 進行 RAG 評估，包括合成評估資料


In [1]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [2]:
!pip install openai
!pip install braintrust autoevals
!pip install langsmith
!pip install pymupdf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# 初始化 Braintrust 和 OpenAI 客戶端
from braintrust import init_logger, traced, wrap_openai, Eval
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')
braintrust_api_key = os.getenv('BRAINTRUST_API_KEY')

# 初始化 Braintrust logger 和 OpenAI client
logger = init_logger(project="Course-202504", api_key=braintrust_api_key)
client = wrap_openai(OpenAI(api_key=openai_api_key))


## 如果使用 colab 的話，執行以下程式碼

In [4]:
# braintrust_api_key = userdata.get('braintrust_api_key')

# import os
# os.environ['OPENAI_API_KEY'] = openai_api_key

# from braintrust import init_logger, traced, wrap_openai, Eval
# from openai import OpenAI

# logger = init_logger(project="Course-202504", api_key=braintrust_api_key)
# client = wrap_openai(OpenAI(api_key=openai_api_key))



## 評估資料準備: 合成資料 Synthetic Data

若要人工製作 dataset，雖然品質好，但是實在太太辛苦了，可以怎麼辦?

可用合成 dataset 策略! 用 LLM 幫我們產生評估資料集

我們用合成的，原理是:

1. 針對要做 RAG 的文本先拆 chunks，也就是 contexts
2. 針對 context 用 LLM 產生對應的 "問題" 和 "參考答案"

more: https://ihower.tw/notes/%E6%8A%80%E8%A1%93%E7%AD%86%E8%A8%98-AI/Synthetic+Data+%E5%90%88%E6%88%90%E8%B3%87%E6%96%99

In [5]:
!wget https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/114/1140224.pdf

--2025-07-07 16:09:50--  https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/114/1140224.pdf
Resolving www.megabank.com.tw (www.megabank.com.tw)... 23.49.116.35
Connecting to www.megabank.com.tw (www.megabank.com.tw)|23.49.116.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1852669 (1.8M) [application/pdf]
Saving to: ‘1140224.pdf.2’

1140224.pdf.2       100%[===================>]   1.77M  10.7MB/s    in 0.2s    

2025-07-07 16:09:50 (10.7 MB/s) - ‘1140224.pdf.2’ saved [1852669/1852669]



In [6]:
import fitz # PyMuPDF library
pages = fitz.open("1140224.pdf")

In [7]:
from typing import List
from pydantic import Field, BaseModel

class QAPair(BaseModel):
    reference: str = Field(..., description="The exact text segment from the original context that this Q&A is based on")
    question: str = Field(description="A single question about the content")
    answer: str = Field(..., description="Answer")

class QAPairs(BaseModel):
    pairs: List[QAPair] = Field(..., description="List of question/answer pairs")

@traced
def produce_questions(content):
    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Please generate 2 question/answer pairs from the following text, focusing specifically on investment and personal finance topics.
                For each pair, provide a single question, a unique answer, and include the exact text segment from the original context that the Q&A is based on.

                IMPORTANT:
                1. Focus ONLY on investment, financial planning, wealth management, stock market, retirement planning, tax optimization, or other personal finance related topics.
                2. All questions and answers MUST be in Traditional Chinese (Taiwan).
                3. Use terminology and expressions commonly used in Taiwan's financial sector.
                4. If the context doesn't contain finance-related information, extract the most relevant aspects that could be applied to personal finance or investment decisions.
                5. For each Q&A pair, include the exact text from the original context that contains the information used for the Q&A. This should be copied verbatim from the input context.

                Context: <context>{content}</context>""",
            }
        ],
        response_format=QAPairs
    )

    parsed_result = completion.choices[0].message.parsed
    pairs = parsed_result.pairs
    return pairs

In [8]:
x = produce_questions('兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現')
x

[QAPair(reference='兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷', question='兆豐銀行提供的投資意見是基於什麼條件進行判斷的？', answer='兆豐銀行的投資意見與市場分析結果是根據當時可取得的資料狀況進行判斷。'),
 QAPair(reference='投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', question='為什麼投資人的投資標的價格與收益會有所變動？', answer='因市場變化影響，投資標的的價格與收益會隨時變動，且不一定代表未來的績效表現。')]

In [9]:
h = x[0].model_dump()

In [10]:
h['test'] = 1234
h

{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷',
 'question': '兆豐銀行提供的投資意見是基於什麼條件進行判斷的？',
 'answer': '兆豐銀行的投資意見與市場分析結果是根據當時可取得的資料狀況進行判斷。',
 'test': 1234}

In [11]:
dataset = []
for idx, page in enumerate(pages):
    context = page.get_text()
    pairs = produce_questions(context)
    for pair in pairs:
      h = pair.model_dump()
      h["page_index"] = idx
      print(h)
      dataset.append(h)

{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '兆豐銀行如何說明其投資意見與市場分析結果的性質？', 'answer': '兆豐銀行的投資意見與市場分析結果是基於當時資料進行判斷，且可能因市場變化而隨時變動，投資標的的價格與收益不代表未來績效。', 'page_index': 0}
{'reference': '財富管理處投顧小組\n114年2月24日\n投資研究週報\n淨零轉型兆豐同行', 'question': '兆豐銀行財富管理處投顧小組的投資研究週報主要探討何種議題？', 'answer': '投資研究週報聚焦於淨零轉型，代表兆豐銀行關注環境永續與相關投資機會。', 'page_index': 0}
{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '兆豐銀行在提供投資意見時，有哪些風險提醒？', 'answer': '兆豐銀行提醒，其投資意見與市場分析是根據資料當時情況判斷，價格與收益會隨市場變動，且不保證未來績效。', 'page_index': 1}
{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '投資標的的價格與收益為何會隨時間波動？', 'answer': '因市場變化，投資標的之價格與收益會隨時變動，導致未來績效不一定與當時分析結果相同。', 'page_index': 1}
{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '在接受兆豐銀行的投資建議時，投資人應該注意什麼重要事項？', 'answer': '投資人應注意兆豐銀行的投資意見與市場分析是基於當時資料判斷，且投資標的價格與收益隨時可能因市場

In [12]:
eval_dataset = []
for qa in dataset:
    eval_dataset.append(
        {
            "input": qa['question'],
            "expected": qa['answer'],
            "metadata": {
                "reference": qa['reference'],
                'page_index': qa['page_index'],
                'file_name': '1140224.pdf'
            },
        }
    )

In [13]:
eval_dataset

[{'input': '兆豐銀行如何說明其投資意見與市場分析結果的性質？',
  'expected': '兆豐銀行的投資意見與市場分析結果是基於當時資料進行判斷，且可能因市場變化而隨時變動，投資標的的價格與收益不代表未來績效。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現',
   'page_index': 0,
   'file_name': '1140224.pdf'}},
 {'input': '兆豐銀行財富管理處投顧小組的投資研究週報主要探討何種議題？',
  'expected': '投資研究週報聚焦於淨零轉型，代表兆豐銀行關注環境永續與相關投資機會。',
  'metadata': {'reference': '財富管理處投顧小組\n114年2月24日\n投資研究週報\n淨零轉型兆豐同行',
   'page_index': 0,
   'file_name': '1140224.pdf'}},
 {'input': '兆豐銀行在提供投資意見時，有哪些風險提醒？',
  'expected': '兆豐銀行提醒，其投資意見與市場分析是根據資料當時情況判斷，價格與收益會隨市場變動，且不保證未來績效。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現',
   'page_index': 1,
   'file_name': '1140224.pdf'}},
 {'input': '投資標的的價格與收益為何會隨時間波動？',
  'expected': '因市場變化，投資標的之價格與收益會隨時變動，導致未來績效不一定與當時分析結果相同。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現',
   'page_index': 1,
   'file_name': '

## Baseline (No RAG)

In [14]:
def simple_qa(question):
    completion = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
    )
    return completion.choices[0].message.content

In [15]:
import autoevals

Eval(
    name="20250707-eval",
    experiment_name="No RAG",
    data=eval_dataset,
    task=simple_qa,
    scores=[autoevals.Factuality(model="gpt-4.1")],
)

Experiment No RAG is running at https://www.braintrust.dev/app/ispan/p/20250707-eval/experiments/No%20RAG
`Eval()` was called from an async context. For better performance, it is recommended to use `await EvalAsync()` instead.


<Task pending name='Task-5' coro=<_EvalCommon.<locals>.run_to_completion() running at /home/os-sunnie.gd.weng/.local/lib/python3.10/site-packages/braintrust/framework.py:688>>

20250707-eval [experiment_name=No RAG] (data): 32it [00:00, 66313.11it/s]


20250707-eval [experiment_name=No RAG] (tasks):   0%|          | 0/32 [00:00<?, ?it/s]


=========================SUMMARY=========================
45.00% 'Factuality' score

1751875914.46s start
1751875935.71s end
13.27s duration
8.39s llm_duration
33.34tok prompt_tokens
423.97tok completion_tokens
457.31tok total_tokens
0.00$ estimated_cost
0tok prompt_cached_tokens
0tok prompt_cache_creation_tokens

See results for No RAG at https://www.braintrust.dev/app/ispan/p/20250707-eval/experiments/No%20RAG


## Naive RAG

我們用 braintrust 提供的 autoevals 套件中的內建評估 prompt: Factuality  https://github.com/braintrustdata/autoevals/blob/main/templates/factuality.yaml

這讓 AI 拿你 task 的 output 和 dataset 中的 expected，比較兩個回答之間的事實內容差異，常見於自動評估問答或教學模型輸出的正確性。以下是詳細解釋：

* (A)	提交的回答是專家答案的子集合，且完全一致。（例：專家說 A+B+C，提交只說 A+C，但沒說錯）
* (B)	提交的回答是專家答案的超集合，也就是它包含更多細節，但與專家內容一致。（例：專家說 A+C，提交說 A+B+C+D）
* (C)	提交的回答與專家答案完全一樣，所有細節一致
* (D)	兩者有事實上的衝突或矛盾
* (E)	雖然兩者不完全一樣，但差異不影響事實正確性（可能是換句話說、順序不同等）

----
* C 與 E 是「最理想」的狀態，得分是 1
* B 比 A 更完整，分數略高。0.6 跟 0.4
* D 是錯誤，得 0 分。




In [16]:
# !pip install chromadb

In [17]:

# Try to fix SQLite compatibility for ChromaDB
import sys
try:
    # Try to override sqlite3 with pysqlite3
    import pysqlite3
    sys.modules['sqlite3'] = pysqlite3
    print("Successfully overridden sqlite3 with pysqlite3")
except ImportError:
    print("pysqlite3 not available, using system sqlite3")
    pass

import chromadb
# Use EphemeralClient which is more compatible
chroma_client = chromadb.EphemeralClient()

# 如何刪除已創建的 collection

# 1. 查看所有現有的 collections
print("現有的 collections:")
collections = chroma_client.list_collections()
for col in collections:
    print(f"- {col.name}")

# 2. 刪除特定的 collection
try:
    chroma_client.delete_collection(name="collection1")
    print("\\n成功刪除 collection1")
except Exception as e:
    print(f"\\n刪除失敗: {e}")

# 3. 再次查看所有 collections 確認是否已刪除
print("\\n刪除後的 collections:")
collections = chroma_client.list_collections()
for col in collections:
    print(f"- {col.name}")
    
# 4. 重新創建 collection（如果需要）
collection = chroma_client.create_collection(name="collection1")



Successfully overridden sqlite3 with pysqlite3
現有的 collections:
\n刪除失敗: Collection collection1 does not exist.
\n刪除後的 collections:


In [18]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [19]:
!pip install langchain_text_splitters

Defaulting to user installation because normal site-packages is not writeable


In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding("o200k_base") # gpt-4o 是 o200k_base，之前版本 gpt-4-turbo 和 gpt-3.5-turbo 是 cl100k_base

def length_function(text: str):
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(length_function=length_function, chunk_size=800, chunk_overlap=200, separators=[
    "\n\n",
    "\n",
    " ",
    ".",
    ",",
    "\u200b",  # Zero-width space
    "\uff0c",  # Fullwidth comma ，
    "\u3001",  # Ideographic comma 、
    "\uff0e",  # Fullwidth full stop ．
    "\u3002",  # Ideographic full stop 。
    "",
])

In [21]:
def get_embeddings(text):
  response = client.embeddings.create(
      input=text,
      model="text-embedding-3-small"
  )

  return response.data[0].embedding

In [22]:
for idx,page in enumerate(pages):
  chunks = text_splitter.split_text(page.get_text())

  collection.add(
    documents = chunks,
    embeddings = [ get_embeddings(chunk) for chunk in chunks ],
    ids=[f"doc-1-page-{idx}-chunk-{x}" for x in range( len(chunks) ) ],
    metadatas=[{"page": idx, "chunk": x} for x in range( len(chunks) ) ]
  )


In [23]:
preview = collection.peek()
print("前幾筆資料預覽：")
print(f"IDs: {preview['ids']}")
print(f"Documents: {preview['documents']}")
print(f"Metadatas: {preview['metadatas']}")

前幾筆資料預覽：
IDs: ['doc-1-page-0-chunk-0', 'doc-1-page-1-chunk-0', 'doc-1-page-2-chunk-0', 'doc-1-page-2-chunk-1', 'doc-1-page-3-chunk-0', 'doc-1-page-4-chunk-0', 'doc-1-page-5-chunk-0', 'doc-1-page-6-chunk-0', 'doc-1-page-7-chunk-0', 'doc-1-page-8-chunk-0']
Documents: ['兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現\n1\n財富管理處投顧小組\n114年2月24日\n投資研究週報\n淨零轉型兆豐同行', '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現\n2\n市場回顧\n市場焦點\n聚焦議題\n資產觀點\n一\n二\n三\n四\n內容大綱', '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現\n3\n市\n場\n焦\n點\n聚\n焦\n議\n題\n資\n產\n觀\n點\n市\n場\n回\n顧\n資料來源：Bloomberg，2025/2/21\n全球主要股票指數(%)\n全球主要債券指數(%)\n全球主要商品指數(%)\n全球主要匯率(%)\n主要金融市場回顧\n股票指數\n過去1個月\n漲跌幅%\n過去1年\n漲跌幅%\n年初迄今\n漲跌幅%\nMSCI AC世界指數\n1.20%\n16.93%\n3.95%\nS&P 500指數\n-0.60%\n20.70%\n2.24%\n歐洲道瓊600指數\n5.30%\n12.79%\n9.11%\nMSCI亞太不含日本指數\n5.31%\n15.02%\n5.72%\n日經225指數\n-0.64%\n1.35%\n-2.80%\nMSCI新興市場指數\n6.12%\n12.41%\n6.68%\n漲跌幅\n週漲跌幅%\n-1.10%\n-1

In [24]:
from typing import List
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    relevant_quotes: List[str]
    answer: str
    following_questions: List[str]

@traced
def ask_with_rag(question):
    results = collection.query(
        query_embeddings = get_embeddings(question),
        # 可有 where 參數可針對上述的 metadatas 做過濾，例如日期、頁數等
        n_results=10
    )

    documents = results['documents'][0]
    context = '\n'.join('* ' + doc for doc in documents)

    user_prompt = f"""
    I will provide you with a document and then ask you a question about it. Please respond following these steps:

    <document>
    {context}
    </document>

    Question: {question}

    Please answer in the following format:

    1. First, identify the most relevant quotes from the document that help answer the question and list them. Each quote should be relatively short.
        If there are no relevant quotes, write "No relevant quotes".

    2. Then, answer the question using facts from these quotes without directly referencing the content in your answer.

    3. Finally, provide 3 related follow-up questions based on the original question and document content that would help explore the topic further.

    If the document does not contain sufficient information to answer the question, please state this in the answer field, but still provide any relevant quotes (if available) and possible follow-up questions.
    Please respond in Traditional Chinese (Taiwan).
    """

    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        messages=[
            {"role": "user", "content": user_prompt},
        ],
        response_format=QueryResult
    )

    parsed_result = completion.choices[0].message.parsed
    return parsed_result.answer

In [ ]:
import autoevals

Eval(
    name="20250707-eval",
    experiment_name="Naive RAG",
    data=eval_dataset,
    task=ask_with_rag,
    scores=[autoevals.Factuality(model="gpt-4.1")],
)


Experiment Naive RAG is running at https://www.braintrust.dev/app/ispan/p/20250707-eval/experiments/Naive%20RAG


<Task pending name='Task-102' coro=<_EvalCommon.<locals>.run_to_completion() running at /home/os-sunnie.gd.weng/.local/lib/python3.10/site-packages/braintrust/framework.py:688>>

20250707-eval [experiment_name=Naive RAG] (data): 32it [00:00, 55438.96it/s]


20250707-eval [experiment_name=Naive RAG] (tasks):   0%|          | 0/32 [00:00<?, ?it/s]


=========================SUMMARY=========================
Naive RAG compared to No RAG:
58.75% (+13.75%) 'Factuality' score	(9 improvements, 2 regressions)

1751875931.54s start
1751875949.34s end
10.92s (-235.21%) 'duration'                    	(18 improvements, 14 regressions)
3.51s (-488.05%) 'llm_duration'                	(29 improvements, 3 regressions)
5556.12tok (+552278.12%) 'prompt_tokens'               	(0 improvements, 32 regressions)
412.66tok (-1131.25%) 'completion_tokens'           	(16 improvements, 15 regressions)
5968.78tok (+551146.88%) 'total_tokens'                	(0 improvements, 32 regressions)
0.00$ (+00.20%) 'estimated_cost'              	(0 improvements, 32 regressions)
476tok (+47600.00%) 'prompt_cached_tokens'        	(9 improvements, 0 regressions)
0tok (-) 'prompt_cache_creation_tokens'	(0 improvements, 0 regressions)

See results for Naive RAG at https://www.braintrust.dev/app/ispan/p/20250707-eval/experiments/Naive%20RAG


## 使用 Ragas 指標來做評估

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

參考自 https://www.braintrust.dev/docs/cookbook/recipes/SimpleRagas

# Ragas Metrics Overview

本文件整理了 Ragas 提供的所有指標，包含描述及適合的場景。

# Ragas Metrics with Real Cases

本文件整理了 Ragas 的指標，包含描述、適合場景以及具體案例。

| **分類**                   | **指標名稱**                     | **描述**                                                                                     | **適合場景**                                                                 | **實際案例**                                                                 |
|----------------------------|----------------------------------|---------------------------------------------------------------------------------------------|------------------------------------------------------------------------------|------------------------------------------------------------------------------|
| **Retrieval Augmented Generation** | Context Precision             | 評估檢索到的上下文是否精準。                                                                | RAG 工作流中需要檢測檢索準確性的場景。                                       | 在法律文件檢索中，檢測是否檢索到了與案件相關的條款。                         |
|                            | Context Recall                 | 評估檢索到的上下文是否完整。                                                                | RAG 工作流中需要檢測檢索完整性的場景。                                       | 在醫學文獻檢索中，檢測是否覆蓋了所有相關的研究。                             |
|                            | Context Entities Recall        | 評估檢索上下文中的實體是否被正確覆蓋。                                                      | 需要檢測實體覆蓋率的場景。                                                   | 在金融報告中，檢測是否覆蓋了所有公司名及相關指標。                           |
|                            | Noise Sensitivity              | 評估模型對噪音的敏感程度。                                                                  | 測試模型在噪音環境下的穩定性。                                               | 測試模型在含有拼寫錯誤或不相關信息的查詢中的表現。                            |
|                            | Response Relevancy             | 評估模型生成的回應是否與上下文相關。                                                        | 需要檢測回應相關性的場景。                                                   | 在客服場景中，檢測回答是否與客戶問題相關。                                   |
|                            | Faithfulness                   | 評估模型生成的回應是否忠實於上下文。                                                        | 測試模型回應的準確性和忠實性。                                               | 在技術支持場景中，檢測回答是否忠實於技術文檔。                               |
|                            | Multimodal Faithfulness        | 評估多模態回應是否忠實於上下文。                                                            | 涉及多模態輸入的場景。                                                       | 在包含圖像和文本的醫療診斷場景中，檢測診斷是否基於正確的輸入。               |
|                            | Multimodal Relevance           | 評估多模態回應是否與上下文相關。                                                            | 涉及多模態輸入的場景。                                                       | 在電子商務場景中，檢測推薦的商品是否與圖片及描述相關。                       |
| **Nvidia Metrics**         | Answer Accuracy                | 評估回答是否準確。                                                                          | 需要檢測回答正確性的場景。                                                   | 在問答系統中，檢測回答是否正確，如「地球的直徑是多少？」                     |
|                            | Context Relevance              | 評估上下文是否與用戶查詢相關。                                                              | 測試上下文相關性。                                                           | 在新聞檢索場景中，檢測提供的新聞是否與查詢主題相關。                         |
|                            | Response Groundedness          | 評估生成的回應是否基於檢索到的上下文。                                                      | 測試回應是否依賴檢索結果的場景。                                             | 在學術文獻生成場景中，檢測摘要是否基於檢索到的文獻。                         |
| **Agentic Workflows**      | Topic Adherence                | 評估模型是否遵守指定的主題。                                                                | 測試模型在特定主題下的表現。                                                 | 在教育場景中，檢測生成的課程內容是否符合指定主題。                           |
|                            | Tool Call Accuracy             | 評估模型使用工具的準確性。                                                                  | 涉及工具調用的場景。                                                         | 在數據分析場景中，檢測模型是否正確調用統計工具進行分析。                     |
|                            | Agent Goal Accuracy            | 評估代理是否達成指定目標。                                                                  | 測試代理目標完成情況。                                                       | 在自動化客服場景中，檢測代理是否成功解決客戶問題。                           |
| **Natural Language Comparison** | Factual Correctness           | 評估回應是否事實正確。                                                                      | 測試事實性和準確性。                                                         | 在百科問答場景中，檢測回答是否符合事實，如「愛因斯坦的出生年份是？」          |
|                            | Semantic Similarity            | 評估回應與目標文本的語義相似度。                                                            | 檢測語義相似性的場景。                                                       | 在翻譯場景中，檢測翻譯文本是否與原文語義一致。                               |
|                            | Non LLM String Similarity      | 使用非 LLM 方法評估字串相似度。                                                             | 需要快速檢測字串相似度的場景。                                               | 在文檔比對中，檢測兩份文檔的字串相似度。                                     |
|                            | BLEU Score                     | 使用 BLEU 方法評估文本相似度。                                                              | 文本生成任務中檢測相似度。                                                   | 在機器翻譯場景中，檢測生成翻譯的質量。                                       |
|                            | ROUGE Score                    | 使用 ROUGE 方法評估文本相似度。                                                             | 文本生成任務中檢測相似度。                                                   | 在摘要生成場景中，檢測生成摘要的質量。                                       |
|                            | String Presence                | 檢測特定字串是否存在。                                                                      | 需要檢測特定字串的場景。                                                     | 在法律合同生成場景中，檢測是否包含必要條款。                                 |
|                            | Exact Match                    | 評估回應是否與目標文本完全匹配。                                                            | 需要檢測完全匹配的場景。                                                     | 在數據表格生成場景中，檢測生成的表格是否與目標表格一致。                     |
| **SQL**                    | Execution based Datacompy Score | 基於執行結果評估 SQL 查詢的準確性。                                                         | 測試 SQL 查詢結果是否準確。                                                  | 在數據庫查詢場景中，檢測查詢結果是否正確。                                   |
|                            | SQL Query Equivalence          | 評估 SQL 查詢是否語義等價。                                                                 | 測試 SQL 語義等價性。                                                        | 在數據遷移場景中，檢測新舊查詢是否語義一致。                                 |
| **General Purpose**        | Aspect Critic                  | 根據指定維度評估模型表現。                                                                  | 自定義評估維度的場景。                                                       | 在產品評論生成場景中，檢測是否符合指定的評論維度。                           |
|                            | Simple Criteria Scoring        | 根據簡單標準評分。                                                                          | 快速評估模型表現的場景。                                                     | 在快速測試場景中，檢測生成文本是否符合基本要求。                             |
|                            | Rubrics Based Scoring          | 使用特定標準評估模型表現。                                                                  | 自定義標準的場景。                                                           | 在教育場景中，根據打分標準評估學生回答。                                     |
|                            | Instance Specific Rubrics Scoring | 根據特定實例的標準評估模型表現。                                                            | 需要針對特定實例進行評估的場景。                                             | 在客製化場景中，根據客戶需求評估生成結果。                                   |
| **Other Tasks**            | Summarization                  | 評估摘要生成的質量。                                                                        | 測試文本摘要生成的場景。                                                     | 在新聞摘要生成場景中，檢測摘要是否準確且全面。                               |

此表格補充了實際案例，幫助理解各指標的應用方式及其在不同場景中的具體使用情況。

In [27]:
@traced
def fetch_top_k_relevant_sections(question):
  results = collection.query(
      query_embeddings = get_embeddings(question),
      # 可有 where 參數可針對上述的 metadatas 做過濾，例如日期、頁數等
      n_results=10
  )

  documents = results['documents'][0]
  return documents

@traced
def generate_answer_from_docs(question, retrieved_content):
  context = '\n'.join('* ' + doc for doc in retrieved_content)

  user_prompt = f"""
  I will provide you with a document and then ask you a question about it. Please respond following these steps:

  <document>
  {context}
  </document>

  Question: {question}

  Please answer in the following format:

  1. First, identify the most relevant quotes from the document that help answer the question and list them. Each quote should be relatively short.
    If there are no relevant quotes, write "No relevant quotes".

  2. Then, answer the question using facts from these quotes without directly referencing the content in your answer.

  3. Finally, provide 3 related follow-up questions based on the original question and document content that would help explore the topic further.

  If the document does not contain sufficient information to answer the question, please state this in the answer field, but still provide any relevant quotes (if available) and possible follow-up questions.
  Please respond in Traditional Chinese (Taiwan).
  """

  completion = client.beta.chat.completions.parse(
      model="gpt-4.1-mini",
      messages=[
          {"role": "user", "content": user_prompt},
      ],
      response_format=QueryResult
  )

  parsed_result = completion.choices[0].message.parsed
  return parsed_result

@traced
def generate_answer_e2e(question):
  retrieved_content = fetch_top_k_relevant_sections(question)
  result = generate_answer_from_docs(question, retrieved_content)

  return { "answer": result.answer, "retrieved_docs": retrieved_content }

In [28]:
from braintrust import EvalAsync

from autoevals import AnswerCorrectness, ContextRecall, ContextPrecision, Faithfulness

# Wrap ContextRecall() to propagate along the "answer" and "context" values separately
async def context_recall(output, **kwargs):
    return await ContextRecall(model="gpt-4.1").eval_async(
        output=output["answer"], context=output["retrieved_docs"], **kwargs
    )

async def context_precision(output, **kwargs):
    return await ContextPrecision(model="gpt-4.1").eval_async(
        output=output["answer"], context=output["retrieved_docs"], **kwargs
    )

async def faithfulness(output, **kwargs):
    return await Faithfulness(model="gpt-4.1").eval_async(
        output=output["answer"], context=output["retrieved_docs"], **kwargs
    )

async def answer_correctness(output, **kwargs):
    return await AnswerCorrectness(model="gpt-4.1").eval_async(output=output["answer"], **kwargs)

eval_result = await EvalAsync(
    name="Course-202504",
    experiment_name="Ragas",
    data=eval_dataset,
    task=generate_answer_e2e,
    scores=[context_recall, context_precision, faithfulness, answer_correctness],
    metadata=dict(model='gpt-4.1-mini', top_k=10),
)

Experiment Ragas is running at https://www.braintrust.dev/app/ispan/p/Course-202504/experiments/Ragas
Course-202504 [experiment_name=Ragas] (data): 32it [00:00, 71203.04it/s]


Course-202504 [experiment_name=Ragas] (tasks):   0%|          | 0/32 [00:00<?, ?it/s]

Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: context_recall [TypeError("unsupported operand type(s) for +: 'int' and 'NoneType'")]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]



=========================SUMMARY=========================
Ragas compared to Naive RAG:
62.33% 'AnswerCorrectness' score
100.00% 'ContextPrecision'  score
88.71% 'ContextRecall'     score
87.12% 'Faithfulness'      score

1751875935.79s start
1751875967.86s end
19.74s duration
4.21s llm_duration
5556.12tok prompt_tokens
409.28tok completion_tokens
5965.41tok total_tokens
0.00$ estimated_cost
216tok prompt_cached_tokens
0tok prompt_cache_creation_tokens

See results for Ragas at https://www.braintrust.dev/app/ispan/p/Course-202504/experiments/Ragas
